# Analyzing the effect of proposed homeless sleeping limits in Los Angeles

By [Matt Stiles](https://www.latimes.com/la-bio-matt-stiles-staff.html) and [Ryan Menezes](https://www.latimes.com/staff/ryan-menezes)

This notebook documents the process of examining the effects of proposed sleeping restriction on the homeless population in Los Angeles. The proposed city rules would limit sleeping in 500-foot buffer zones around selected property types, including schools, parks and childcare facilities, among others. This analysis draws those buffer zones around the properties, which are identified in government mapping data, and calculates the area included in the limits citywide and by neighborhood.  

### Load libraries

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pandas import DataFrame
%matplotlib inline

### Read raw data from city, county GIS portals

In [2]:
rawfiles = {
    # childcare spatial file uses points to avoid assigning single-location facilities to multi-location parcels
    # bikeways not included because they don't need to be buffered
    # 'bikeways': 'raw/la_city_bikeways.geojson',
    # homeless shelters excluded until recent locations file acquired
    #'homeless': 'raw/la_county_gis_homeless_shelters_1566703407847.geojson',
    'childcare': 'raw/la_city_gis_childcare_clipped.geojson',
    'parks': 'raw/la_city_county_parks.geojson',
    'public_schools': 'raw/lausd_schools_boundaries_1566703408821.geojson',
    'private_schools': 'raw/la_county_private_school_parcels.geojson',
    'special_venues': 'raw/la_city_special_venues.geojson'
}

### Reproject spatial data for distance calculations

In [3]:
def reproject(i, o, c = 3310):
    return f'ogr2ogr processed/{o}_{c}.geojson -t_srs "EPSG:3310" {i}'

In [4]:
for r in rawfiles:
    s = reproject(rawfiles[r], r)
    print(s)
    !$s

ogr2ogr processed/childcare_3310.geojson -t_srs "EPSG:3310" raw/la_city_gis_childcare_clipped.geojson
ogr2ogr processed/parks_3310.geojson -t_srs "EPSG:3310" raw/la_city_county_parks.geojson
ogr2ogr processed/public_schools_3310.geojson -t_srs "EPSG:3310" raw/lausd_schools_boundaries_1566703408821.geojson
ogr2ogr processed/private_schools_3310.geojson -t_srs "EPSG:3310" raw/la_county_private_school_parcels.geojson
ogr2ogr processed/special_venues_3310.geojson -t_srs "EPSG:3310" raw/la_city_special_venues.geojson


### Dataframes from city, county spatial files

In [ ]:
alldata = pd.concat([gpd.read_file(f'processed/{r}_3310.geojson') for r in rawfiles])[['geometry']]

In [ ]:
alldata.crs

In [ ]:
alldata.head()

In [ ]:
alldata.plot()

In [ ]:
alldata['buffer'] = alldata.buffer(152.4)

In [ ]:
gpd.GeoSeries(alldata['buffer']).plot()

In [ ]:
alldata['dummy'] = 'abc'

In [ ]:
alldata.head()

In [ ]:
allbuffers = gpd.GeoDataFrame(alldata[['dummy','buffer']], geometry='buffer').dissolve(by = 'dummy')

In [ ]:
allbuffers.plot()

In [ ]:
allbuffers.to_file('buffered/combined-buffers.geojson', driver='GeoJSON')

In [ ]:
allbuffers.crs

In [ ]:
allbuffers.geometry.area  / 2590000

In [ ]:
hoods = gpd.read_file('raw/la-county-neighborhoods-current.geojson')

In [ ]:
hoods['hood_type'] = hoods['metadata'].apply(dict).apply(lambda x: x['type'])

In [ ]:
hoods.to_crs({'init': 'epsg:3310'}, inplace=True)

In [ ]:
hoods.head()

In [ ]:
lahoods = hoods.loc[hoods.hood_type == 'segment-of-a-city', ['name', 'geometry']]

In [ ]:
lahoods.head()

In [ ]:
lahoods.head()

Puts the buffer shape on each line

In [ ]:
lahoods['buffers'] = lahoods['name'].apply(lambda x: allbuffers['buffer'][0])

In [ ]:
lahoods.head()

In [ ]:
len(lahoods)

In [ ]:
lahoods['hits_buffer'] = lahoods.apply(lambda x: x.geometry.intersects(x.buffers), axis = 1)

How many of the lahoods hit the buffer?

In [ ]:
lahoods['hits_buffer'].sum()

In [ ]:
lahoods['intersection'] = lahoods.apply(lambda x: x.geometry.intersection(x.buffers), axis=1)

In [ ]:
lahoods.head()

In [ ]:
lahoods['hood_area'] = lahoods.geometry.area / 2590000

In [ ]:
lahoods['buffer_area'] = lahoods['intersection'].apply(lambda x: x.area / 2590000)

In [ ]:
lahoods['pct_of_hood_in_buffer'] = lahoods.buffer_area / lahoods.hood_area

In [ ]:
lahoods.sort_values('pct_of_hood_in_buffer', ascending=False)

In [ ]:
intersection_gdf = gpd.GeoDataFrame(lahoods[['name','intersection', 'pct_of_hood_in_buffer']], geometry = 'intersection')

In [ ]:
intersection_gdf.head()

### Reproject for web mapping

In [ ]:
intersection_gdf.head()

In [ ]:
allbuffers.head()

In [ ]:
intersection_gdf.crs = {'init' :'epsg:3310'}
allbuffers.crs = {'init' :'epsg:3310'}

In [ ]:
intersection_gdf.to_crs({'init': 'epsg:4326'}, inplace=True)
allbuffers.to_crs({'init': 'epsg:4326'}, inplace=True)

In [ ]:
allbuffers.head()

### Export spatial file to json file

In [ ]:
intersection_gdf.to_file('buffered/buffers-by-hood.geojson', driver='GeoJSON')
allbuffers.to_file('buffered/combined-buffers.geojson', driver='GeoJSON')

### Export CSV for reporting on neighborhoods

In [ ]:
export_hood_pct_table = pd.DataFrame(lahoods[['name','hood_area', 'buffer_area','pct_of_hood_in_buffer']]).sort_values('pct_of_hood_in_buffer', ascending=False)

In [ ]:
export_hood_pct_table.to_csv('buffered/hood-breakdown.csv')

### Create mdtiles for Mapbox

`tippecanoe -o combined-buffers.mbtiles -zg -pt --coalesce-densest-as-needed --extend-zooms-if-still-dropping --generate-ids --projection=EPSG:4326 combined-buffers.geojson`